In [1]:
from neg_data4 import *
if __name__ == '__main__':
    print('Loading data...')
    prod_to_reacs = defaultdict(set)
    prod_to_noreacs = defaultdict(set)
    Tolri=0
    Tolpos=0
    Tolneg=0
    batch_size = 1
#    reactant = []
#    rule_ans = []
#    prod_ans = []

#    expansion_rules = []
    tem_simp = set()
#    seed =0
#    random.seed(seed)
#    with open('data/templates_expansion3.dat', 'r') as f:
#        for l in f:
#            reactant.append(l.strip().split('\t')[2])
#            rule_ans.append(l.strip().split('\t')[0])
#            prod_ans.append(l.strip().split('\t')[1])
#    batch_datasX = reactant[:10]
#    batch_datasY = rule_ans[:10]
#    batch_datasZ = prod_ans[:10]
#    with open('data/expansion_expansion.dat', 'r') as f:
#        for i, l in tqdm(enumerate(f), desc='expansion'):
#            rule = l.strip()
#            expansion_rules.append(rule)

#    with open('data/templates_expansion1.dat', 'r') as f:
#        for l in tqdm(f, desc='products'):
#                tem_simp.add(l.strip())
#    with open('data/templates_expansion3.dat', 'w') as f:
#        f.write('\n'.join(tem_simp))                           

    with open('data/templates_expansion3.dat', 'r') as f:
    #    for l in tqdm(f, desc='products'):
#        seeds=[a for a in range(len(f.readlines()))]
#        combo[0]=f
#        combo[1]=seeds
    

    
        with Pool() as p:
#            for reacs,noreacs in tqdm(p.imap(partial(read, rules=expansion_rules), f)):
            for reacs,noreacs in tqdm(p.imap(read, zip(f, seedgen(batch_size)))):  
                for reac, values in reacs.items():
                    for value in values:
                        prod_to_reacs[reac].add(value)
                
                for reac, values in noreacs.items():
                    for value in values:
                        prod_to_noreacs[reac].add(value)
                    
    transforms=[]  
    for prod, reacs in prod_to_reacs.items():
        for reac in reacs:
            transforms.append((prod, reac, '1'))
            Tolpos+=1

    for prod, noreacs in prod_to_noreacs.items():
        for noreac in noreacs:
            transforms.append((prod, noreac, '0'))             
            Tolneg+=1
    with open('data/inscopedata4all.dat', 'w') as f:
        f.write('\n'.join(['\t'.join(rxn_prod) for rxn_prod in transforms]))

    print('total positive Expansion rules examples:',Tolpos) 
    print('total negative Expansion rules examples:',Tolneg)

RDKit WARNING: [19:46:04] Enabling RDKit 2019.09.3 jupyter extensions
Using TensorFlow backend.
expansion: 176756it [00:00, 726352.51it/s]


Loading data...


0it [00:00, ?it/s]

Instructions for updating:
Colocations handled automatically by placer.


1157756it [5:24:00, 59.55it/s]  


total positive Expansion rules examples: 430665
total negative Expansion rules examples: 22857611


In [1]:
from neg_data4 import *



if __name__ == '__main__':
    print('Loading data...')
    prod_to_reacs = defaultdict(set)
    prod_to_noreacs = defaultdict(set)
    Tolri=0
    Tolpos=0
    Tolneg=0
    reactant = []
    rule_ans = []
    prod_ans = []

#    expansion_rules = []
    tem_simp = set()
#    seed =0
#    random.seed(seed)
    save_dir = os.path.join(os.getcwd(), 'saved_models')
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    
    model_path_e = os.path.join(save_dir, 'trained_model_negative_1')
    fp_dim=23086
    idx = np.load(os.path.join(save_dir, 'expansion5e-5.idx.npy'))   

    
    negative_net = load_model(model_path_e, custom_objects={'acc_top10': acc_top10,'acc_top50': acc_top50, 'Highway': Highway})   
    with open('data/templates_expansion3.dat', 'r') as f:
        for l in f:
            reactant.append(l.strip().split('\t')[2])
            rule_ans.append(l.strip().split('\t')[0])
            prod_ans.append(l.strip().split('\t')[1])
    batch_datasX = reactant[:10]
    batch_datasY = rule_ans[:10]
    batch_datasZ = prod_ans[:10]
#    with open('data/expansion_expansion.dat', 'r') as f:
#        for i, l in tqdm(enumerate(f), desc='expansion'):
#            rule = l.strip()
#            expansion_rules.append(rule)

#    with open('data/templates_expansion1.dat', 'r') as f:
#        for l in tqdm(f, desc='products'):
#                tem_simp.add(l.strip())
#    with open('data/templates_expansion3.dat', 'w') as f:
#        f.write('\n'.join(tem_simp))                           

#    with open('data/templates_expansion3.dat', 'r') as f:
    #    for l in tqdm(f, desc='products'):
#        seeds=[a for a in range(len(f.readlines()))]
#        combo[0]=f
#        combo[1]=seeds
    if True:
        X = np.zeros((len(batch_datasX),fp_dim)) 
        for i,a in enumerate(batch_datasX):
            n = np.zeros((1,fp_dim))        
            for b in smi_list_from_str(a):
                n += preprocess([b], fp_dim, idx) 
            X[i] = n
        X = np.log(X+1)
        # Predict applicable rules
        predict = negative_net.predict_on_batch(X)
        # get the rules index from high probability to low probability
        preds =np.argsort(-predict, axis=1)[:,:50]
    
        with Pool(processes = 2) as p:
#            for reacs,noreacs in tqdm(p.imap(partial(read, rules=expansion_rules), f)):
            for reacs,noreacs in tqdm(p.imap(read, zip(batch_datasY, batch_datasZ, batch_datasX, preds))):  
                for reac, values in reacs.items():
                    for value in values:
                        prod_to_reacs[reac].add(value)
                
                for reac, values in noreacs.items():
                    for value in values:
                        prod_to_noreacs[reac].add(value)
                    
    transforms=[]  
    for prod, reacs in prod_to_reacs.items():
        for reac in reacs:
            transforms.append((prod, reac, '1'))
            Tolpos+=1

    for prod, noreacs in prod_to_noreacs.items():
        for noreac in noreacs:
            transforms.append((prod, noreac, '0'))             
            Tolneg+=1
    with open('data/inscopedata4.dat', 'w') as f:
        f.write('\n'.join(['\t'.join(rxn_prod) for rxn_prod in transforms]))

    print('total positive Expansion rules examples:',Tolpos) 
    print('total negative Expansion rules examples:',Tolneg)

RDKit WARNING: [20:23:29] Enabling RDKit 2019.09.3 jupyter extensions
Using TensorFlow backend.
expansion: 55608it [00:00, 796212.30it/s]


Loading data...
Instructions for updating:
Colocations handled automatically by placer.


10it [00:02,  1.61s/it]


total positive Expansion rules examples: 1
total negative Expansion rules examples: 73


In [ ]:
Tolpos